# How to use the lookup table to assign mass-to-light ratios and stellar mass densities to data

Isaac Cheng - December 2021

Based on the [`stellar_mass_densities+vorbin_radial_profiles`
notebook](../galaxies/stellar_mass_densities/stellar_mass_densities+vorbin_radial_profiles.ipynb).

Note that there are only 36 galaxies in this analysis because several VERTICO galaxies are
not in the NGVS footprint and I got the NGC 4189 flag map too late and NGC 4606's flag map
is too intrusive.


In [1]:
%cd "/arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities/"  # change directory to where this file is

from multiprocessing import Pool
from itertools import repeat

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dill
import astropy.units as u
# 
# Load my own packages
# 
import sys
sys.path.append("/arc/home/IsaacCheng/coop_f2021/packages")
import fits_utils as fu
import radial_profile_utils as rpu
# 
GALDIST = 16.5 * u.Mpc  # distance to Virgo cluster centre

/arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities


Below are all the galaxies in our NGVS-VERTICO sample


In [2]:
# ----------------------------------------------------------------------------------------
GALAXIES = np.array(
    [
        "IC3392",
        # "NGC4189",  # ignored flag map, but bad results
        "NGC4192",
        "NGC4216",  # has unflagged foreground star
        "NGC4222",
        "NGC4254",
        "NGC4294",
        "NGC4298",
        "NGC4299",
        "NGC4302",
        "NGC4321",
        "NGC4330",
        "NGC4351",
        "NGC4380",
        "NGC4383",
        "NGC4388",
        "NGC4396",
        "NGC4402",
        "NGC4405",
        "NGC4419",
        "NGC4424",
        "NGC4450",
        "NGC4501",
        "NGC4522",
        "NGC4532",
        "NGC4535",
        "NGC4548",
        "NGC4567",
        "NGC4568",
        "NGC4569",
        "NGC4579",
        "NGC4580",
        # "NGC4606",  # flag map too intrusive
        "NGC4607",
        "NGC4651",
        "NGC4654",
        "NGC4689",
        "NGC4694",
    ]
)
INCLINATIONS = np.array(
    [
        68,
        # 42,  # ignored flag map, but bad results
        83,
        90,  # has unflagged foreground star
        90,
        39,
        74,
        52,
        14,
        90,
        32,
        90,
        48,
        61,
        56,
        83,
        83,
        80,
        46,
        74,
        61,
        51,
        65,
        82,
        64,
        48,
        37,
        49,
        70,
        69,
        40,
        46,
        # 69,  # flag map too intrusive
        90,
        53,
        61,
        38,
        62,
    ]
)  # degrees
POSITION_ANGLES = np.array(
    [
        219,
        # 70,  # ignored flag map, but bad results
        333,
        20,  # has unflagged foreground star
        238,
        243,
        151,
        132,
        128,
        356,
        280,
        238,
        251,
        158,
        17,
        271,
        304,
        270,
        18,
        131,
        274,
        170,
        320,
        35,
        159,
        12,
        318,
        251,
        211,
        203,
        273,
        337,
        # 38,  # flag map too intrusive
        2,
        75,
        300,
        341,
        323,
    ]
)  # degrees
# ----------------------------------------------------------------------------------------
VCC_GALAXIES = np.array(  # the VCC number of each galaxy
    [
        1126,  # IC3392
        # 89,  # NGC4189, ignored flag map, but bad results
        92,  # NGC4192
        167,  # NGC4216, has unflagged foreground star
        187,  # NGC4222
        307,  # NGC4254
        465,  # NGC4294
        483,  # NGC4298
        491,  # NGC4299
        497,  # NGC4302
        596,  # NGC4321
        630,  # NGC4330
        692,  # NGC4351
        792,  # NGC4380
        801,  # NGC4383
        836,  # NGC4388
        865,  # NGC4396
        873,  # NGC4402
        874,  # NGC4405
        958,  # NGC4419
        979,  # NGC4424
        1110,  # NGC4450
        1401,  # NGC4501
        1516,  # NGC4522
        1554,  # NGC4532
        1555,  # NGC4535
        1615,  # NGC4548
        1673,  # NGC4567
        1676,  # NGC4568
        1690,  # NGC4569
        1727,  # NGC4579
        1730,  # NGC4580
        # 1859,  # NGC4606, flag map too intrusive
        1868,  # NGC4607
        -100,  # NGC4651 (EVCC number is 1102, cannot use EVCC number)
        1987,  # NGC4654
        2058,  # NGC4689
        2066,  # NGC4694
    ]
)

## 1. Load NGVS Catalogue

We use the NGVS catalogue to calculate each galaxy's extinction-corrected colours.

M.B. when using the lookup table, all data should be extinction-corrected.


In [3]:
ngvs_catalogue = pd.read_csv("/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS_catalogue.txt", sep=" ", low_memory=False)
ngvs_catalogue

,#Official_name,Old_name,VCC_name,VCC_membership,VCC_Bmag,TH_name,sep,NGVS_ra(deg),NGVS_dec(deg),NGVS_ra(hms),...,MAGERR_GALFIT_z_nuc,RE_GALFIT_z_nuc,REERR_GALFIT_z_nuc,N_GALFIT_z_nuc,NERR_GALFIT_z_nuc,Q_GALFIT_z_nuc,PA_GALFIT_z_nuc,SKY_GALFIT_z,CHI2NU_z,sep.43
0,NGVSJ12:03:45.06+27:37:02.0,NGVSJ12:03:45.06+27:37:02.0,-100,-100,-100.00,-100,||,180.937750,27.617222,12:03:45.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NGVSJ12:04:51.83+26:59:33.1,NGVSJ12:04:51.83+26:59:33.1,-100,-100,-100.00,-100,||,181.215958,26.992528,12:04:51.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NGVSJ12:07:45.73+12:03:37.0,NGVSJ12:07:45.73+12:03:37.0,-100,-100,-100.00,-100,||,181.940552,12.060277,12:07:45.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NGVSJ12:08:01.21+12:48:56.0,NGVSJ12:08:01.21+12:48:56.0,-100,-100,-100.00,-100,||,182.005060,12.815564,12:08:01.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NGVSJ12:08:04.43+13:13:17.0,NGVSJ12:08:04.43+13:13:17.0,-100,-100,-100.00,-100,||,182.018458,13.221389,12:08:04.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,NGVSJ12:52:55.97+11:13:51.5,NGVS193.23325+11.230949_VCC2095,2095,M,11.18,-100,||,193.233227,11.230976,12:52:55.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3685,NGVSJ12:52:59.03+14:24:01.9,NGVSJ12:52:59.03+14:24:01.9,-100,-100,-100.00,-100,||,193.245965,14.400533,12:52:59.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3686,NGVSJ12:53:01.39+13:07:54.3,NGVSJ12:53:01.39+13:07:54.3,-100,-100,-100.00,-100,||,193.255792,13.131739,12:53:01.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3687,NGVSJ12:53:11.59+12:38:06.3,NGVSJ12:53:11.59+12:38:06.3,-100,-100,-100.00,-100,||,193.298303,12.635086,12:53:11.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since NGC 4651 does not have a VCC number, find the entry in the NGVS catalogue that most
closely matches the coordinates of NGC 4651.


In [4]:
ngvs_ra = ngvs_catalogue["NGVS_ra(deg)"]
ngvs_dec = ngvs_catalogue["NGVS_dec(deg)"]
vcc_name = ngvs_catalogue["VCC_name"]
extinctions = ngvs_catalogue["e(B-V)"]

# Single entry that is closest to NGC4651 coordinates
NGC4651_row = vcc_name[
    (abs(ngvs_ra - 190.927625) < 0.1) & (abs(ngvs_dec - 16.39338889) < 0.1)
].index.values[0]

# E(B-V) to u, g, i, z extinction coefficients
# (from http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/query/catdoc.html#ebv)
EBV_u = 4.594
EBV_g = 3.560
EBV_i = 1.813
EBV_z = 1.221

## 2. Load lookup table data as global variables

In [ ]:
LOOKUP_PATH = "/arc/home/IsaacCheng/coop_f2021/galaxies/lookup_table/"  # change this
lookup_table_infile = LOOKUP_PATH + "TOTAL_no4189_extinctionCorr_lookupTable.pkl"  # change this
with open(lookup_table_infile, "rb") as f:
    file = dill.load(f)
    gz_lookup_vals = file["gz_vals"]  # 1D array
    ui_lookup_vals = file["ui_vals"]  # 1D array
    MLi_lookup_vals = file["MLi"]  # 2D array
    MLi_lookup_unc = file["MLi_unc"]  # 2D array
    file = None  # free memory

## 3. General functions to assign mass-to-light ratio & to calculate stellar mass densitites


In [5]:
def assign_MLi(
    gz_color,  # should be extinction corrected
    ui_color,  # should be extinction corrected
    outside_to_closest=True,  # if True, assign values beyond the edges of the lookup table to the closest bin
    print_debug=False,
):
    """
    Uses lookup table (global variable) to assign mass-to-light ratios (MLi) and their
    uncertainties to some data given their extinction-corrected g-z and u-i colours.
    
    This will be called by assign_M_density().
    """
    #
    # Check inputs
    #
    gz_color = np.asarray(gz_color)
    ui_color = np.asarray(ui_color)
    if gz_color.shape != ui_color.shape:
        raise ValueError("gz_color and ui_color must be have the same shape!")
    #
    # Since gz_lookup_vals and ui_lookup_vals are increasing & right=False, the returned
    # index satisfies: bins[i] <= x < bins[i+1] (since I subtracted 1 from the bin number)
    #
    gz_idx = np.digitize(gz_color, gz_lookup_vals, right=False) - 1
    ui_idx = np.digitize(ui_color, ui_lookup_vals, right=False) - 1
    #
    # Handle points outside of lookup table (negative index indicates bad value)
    #
    if outside_to_closest:
        # Assign points beyond g-z and u-i ranges to closest bin
        gz_idx[gz_color <= gz_lookup_vals.min()] = 0
        ui_idx[ui_color <= ui_lookup_vals.min()] = 0
        gz_idx[gz_color >= gz_lookup_vals.max()] = len(gz_lookup_vals) - 1
        ui_idx[ui_color >= ui_lookup_vals.max()] = len(ui_lookup_vals) - 1
    else:
        gz_idx[gz_color <= gz_lookup_vals.min()] = -1
        ui_idx[ui_color <= ui_lookup_vals.min()] = -1
        gz_idx[gz_color >= gz_lookup_vals.max()] = -1
        ui_idx[ui_color >= ui_lookup_vals.max()] = -1
    if print_debug:
        print("Num NaNs after optional outside_to_closest:", np.sum(gz_idx < 0), np.sum(ui_idx < 0))
    #
    # Set bad values to negative index
    #
    gz_idx_mask = ~np.isfinite(gz_color)
    ui_idx_mask = ~np.isfinite(ui_color)
    gz_idx[gz_idx_mask] = -1
    ui_idx[ui_idx_mask] = -1
    if print_debug:
        print("Num NaNs after finite-check:", np.sum(gz_idx < 0), np.sum(ui_idx < 0))
    #
    # Assign MLi and MLi uncertainties
    #
    MLi = np.full(gz_color.shape, np.nan)
    MLi_unc = np.full(gz_color.shape, np.nan)
    for gz_bin in np.unique(gz_idx).astype(int):
        for ui_bin in np.unique(ui_idx).astype(int):
            if gz_bin < 0 or ui_bin < 0:
                continue
            MLi_mask = (gz_idx == gz_bin) & (ui_idx == ui_bin)
            MLi[MLi_mask] = MLi_lookup_vals[gz_bin, ui_bin]
            MLi_unc[MLi_mask] = MLi_lookup_unc[gz_bin, ui_bin]
    if print_debug:
        print("Num NaNs after lookup table:", np.sum(~np.isfinite(MLi)), np.sum(~np.isfinite(MLi_unc)))
    return MLi, MLi_unc


def rel_to_abs_mag(rel_mag, dist, rel_mag_err=0.0, dist_err=0.0):
    """
    Converts relative magnitudes to absolute magnitudes.
    N.B. `dist` and `dist_err` should be in parsecs.
    
    This will be called by assign_M_density().
    """
    abs_mag = rel_mag - 5 * (np.log10(dist) - 1)
    abs_mag_err = np.sqrt(rel_mag_err ** 2 + (5 / np.log(10) * dist_err / dist) ** 2)
    return abs_mag, abs_mag_err


def assign_M_density(  # returns MLi, MLi_err, M_density, M_density_err
    gz_color,  # should be extinction corrected
    ui_color,  # should be extinction corrected
    iband_rel_mag,  # should be extinction corrected and normalized by number of pixels in each bin
    iband_rel_mag_err,  # should be extinction corrected and normalized by number of pixels in each bin
    wcs,  # the WCS object to get physical dimensions of each pixel (for stellar mass densities)
    inclination=None,  # if not None, correct stellar mass density for inclination (degrees)
    dist=16.5 * u.Mpc,  # astropy quantity or scalar in parsecs
    dist_err=0 * u.Mpc,  # astropy quantity or scalar in parsecs
    outside_to_closest=True,  # if True, assign values beyond the edges of the lookup table to the closest bin
    iband_abs_mag_sun=4.53,  # absolute i-band AB magnitude of the Sun (default value from FSPS)
    print_debug=False,
):
    """
    Given some data's g-z colours, u-i colours, and i-band relative magnitudes, estimates
    their mass-to-light ratios (MLi) and stellar mass densities (M_density) as well as
    their respective uncertainties from a lookup table (which is a global variable).

    N.B. All colours and relative magnitudes should be extinction-corrected.
    """
    #
    # Convert distance to consistent unit (parsecs)
    #
    if isinstance(dist, u.Quantity):
        dist = dist.to(u.parsec).value
    if isinstance(dist_err, u.Quantity):
        dist_err = dist_err.to(u.parsec).value
    #
    # Apply lookup table to get mass-to-light ratios
    #
    MLi, MLi_err = assign_MLi(
        gz_color,
        ui_color,
        outside_to_closest=outside_to_closest,
        print_debug=print_debug,
    )
    #
    # Calculate i-band luminosities
    #
    iband_abs_mag, iband_abs_mag_err = rel_to_abs_mag(
        iband_rel_mag, dist, rel_mag_err=iband_rel_mag_err, dist_err=dist_err,
    )
    Li = 10 ** (-0.4 * (iband_abs_mag - iband_abs_mag_sun))  # in solar luminosities
    Li_err = Li * np.log(10) * 0.4 * iband_abs_mag_err
    #
    # Calculate stellar masses
    #
    M = MLi * Li  # in solar masses
    M_err = M * np.sqrt((MLi_err / MLi) ** 2 + (Li_err / Li) ** 2)
    #
    # Convert to stellar mass densities and (optionally) correct for inclination
    #
    px_dimensions, px_dimensions_err = fu.calc_pc_per_px(
        wcs, dist * u.pc, dist_err=dist_err * u.pc
    )  # parsecs
    px_area = px_dimensions[0] * px_dimensions[1]  # square parsecs
    px_area_err = px_area * np.sqrt(np.sum((px_dimensions_err / px_dimensions) ** 2))
    if inclination is not None:
        M_density = rpu.correct_for_i(M / px_area, inclination, i_threshold=80, i_replacement=80)  # correct for inclination
    else:
        M_density = M / px_area
    M_density_err = M_density * np.sqrt((M_err / M) ** 2 + (px_area_err / px_area) ** 2)
    #
    return MLi, MLi_err, M_density, M_density_err  # N.B. stellar mass densities in solar masses per square parsec


## 4. Function to read NGVS data and calculate a plethora of useful quantities (e.g., MLi, stellar mass densities, etc.)

The following function will use NGVS data that have been binned at one of three
resolutions:

1. adaptively binned using Voronoi binning (`bin_resolution=vorbin`)
2. regularly binned to VERTICO's 9" beam with 2" pixel resolution (`bin_resolution=9as`)
3. regularly binned to VERTICO's 9" beam with 4" pixel resolution
   (`bin_resolution=nyquist`)

to calculate a bunch of useful quantities (e.g., extinction corrected magnitudes,
mass-to-light ratios, stellar mass densities). For each galaxy, this function will produce
a pickle file containing all the raw + derived data for this galaxy. The resulting pickle
files will be the basis for pretty much all of the analysis involving optical data or
their derived products.

Finally, you will need to change the pickle file paths!

Other notes:
- The `_vorbin_to_M_density()`
sub-method will use the pickle files produced by
[`vorbin_tutorial.ipynb`](vorbin_tutorial.ipynb) while the `_regBin_to_M_density()`
sub-method will use the pickle files produced by
[`regbin_tutorial.ipynb`](regbin_tutorial.ipynb).
- The `M` variable means "stellar mass" (units of solar masses) and `M_density` means
stellar mass density (units of solar masses per square parsec).
- I have an if statement in the vorbin sub-method to handle NGC 4651 separately (since it
  has no VCC number). You may need to remove or adapt this for other galaxies. Note that
  the regular binning sub-method uses pickle files that contain data that have already
  been extinction-corrected, hence I don't need to re-correct for extinction.


In [6]:

def ngvs_to_MLi(
    ngc_galname,  # the string containing the NGC/IC name (e.g., "NGC4380")
    vcc_galnum,  # the VCC ID from the NGVS catalogue (e.g., 792)
    inclination,  # degrees, to correct stellar mass densities for inclination
    bin_resolution="vorbin",  # "vorbin", "nyquist", or "9as",
    snr_target=50,  # only relevant if bin_resolution is "vorbin". Specifies the vorbin file to use
    good_snr=30,  # minimum SNR for a Voronoi bin to be considered "good". May be useful because of potential overflow errors
    dist=GALDIST,  # the distance to the galaxy. Should be astropy quantity (e.g., 16.5 * u.Mpc)
    outside_to_closest=True,  # if True, assign values beyond the edges of the lookup table to the closest bin
    print_debug=False,
):

    def _vorbin_to_M_density():
        with open(galpath + f"{ngc_galname}_vorbin_SNR{snr_target}_ugizBinned.pkl", "rb") as f:  # ! CHANGE ME
            file = dill.load(f)
            x_coords = file["x_coords"]
            y_coords = file["y_coords"]
            wcs = file["wcs"]
            wcs.array_shape = file["wcs_array_shape"]
            uband_signal = file["uband_signal"]
            uband_noise = file["uband_noise"]
            gband_signal = file["gband_signal"]
            gband_noise = file["gband_noise"]
            iband_signal = file["iband_signal"]
            iband_noise = file["iband_noise"]
            zband_signal = file["zband_signal"]
            zband_noise = file["zband_noise"]
            px_per_bin = file["px_per_bin"]
            binNum = file["binNum"]
            binNum_arr = file["binNum_arr"]
            file = None  # free memory
        #
        # Make mask for pixels with finite SNRs >= good_snr (e.g., good_snr=30)
        # (may be useful in the future. Don't know)
        #
        uband_snr = uband_signal / uband_noise
        gband_snr = gband_signal / gband_noise
        iband_snr = iband_signal / iband_noise
        zband_snr = zband_signal / zband_noise
        isgood_snr = (
            (uband_snr >= good_snr)
            & (gband_snr >= good_snr)
            & (iband_snr >= good_snr)
            & (zband_snr >= good_snr)
            & np.isfinite(uband_snr)
            & np.isfinite(gband_snr)
            & np.isfinite(iband_snr)
            & np.isfinite(zband_snr)
        )
        #
        # Normalize fluxes
        #
        uband_signal = uband_signal / px_per_bin
        uband_noise = uband_noise / px_per_bin
        gband_signal = gband_signal / px_per_bin
        gband_noise = gband_noise / px_per_bin
        iband_signal = iband_signal / px_per_bin
        iband_noise = iband_noise / px_per_bin
        zband_signal = zband_signal / px_per_bin
        zband_noise = zband_noise / px_per_bin
        #
        # Calculate relative AB magnitudes
        #
        uband_rel_mag, uband_rel_mag_err = fu.calc_mag(
            uband_signal, uband_noise, zpt=30.0, calc_abs=False
        )
        gband_rel_mag, gband_rel_mag_err = fu.calc_mag(
            gband_signal, gband_noise, zpt=30.0, calc_abs=False
        )
        iband_rel_mag, iband_rel_mag_err = fu.calc_mag(
            iband_signal, iband_noise, zpt=30.0, calc_abs=False
        )
        zband_rel_mag, zband_rel_mag_err = fu.calc_mag(
            zband_signal, zband_noise, zpt=30.0, calc_abs=False
        )
        #
        # Correct for extinction
        #
        if ngc_galname != "NGC4651":
            ebv_coeff = extinctions[vcc_name == vcc_galnum].values
            if ebv_coeff.size != 1:
                raise ValueError(f"More than one E(B-V) coefficient found for {ngc_galname}")
            ebv_coeff = ebv_coeff[0]
        else:
            ebv_coeff = extinctions.iloc[NGC4651_row]
        uband_rel_mag = uband_rel_mag - ebv_coeff * EBV_u
        gband_rel_mag = gband_rel_mag - ebv_coeff * EBV_g
        iband_rel_mag = iband_rel_mag - ebv_coeff * EBV_i
        zband_rel_mag = zband_rel_mag - ebv_coeff * EBV_z
        # 
        # Calculate mass-to-light ratio and stellar mass density
        # 
        gz_colour = gband_rel_mag - zband_rel_mag
        ui_colour = uband_rel_mag - iband_rel_mag
        MLi, MLi_err, M_density, M_density_err = assign_M_density(
            gz_colour, ui_colour, iband_rel_mag, iband_rel_mag_err, wcs,
            inclination=inclination, dist=dist, outside_to_closest=outside_to_closest,
            print_debug=print_debug,
        )
        # 
        # Pickle data
        # 
        galaxy_outfile = galpath + f"{ngc_galname}_vorbin_SNR{snr_target}_extinctionCorr_ugiz_Sigma-star_i_corr.pkl"  # ! CHANGE ME
        with open(galaxy_outfile, "wb") as f:
            dill.dump(
                {
                    "README": "The Voronoi-binned signal and noise arrays, as well as the relative magnitudes, have all been normalized by the number of pixels per bin. "
                    + "To recover the un-normalized binned signal and noise arrays, simply multiply the signal and noise arrays by `binNum_arr`. "
                    + f"Note that this is in contrast to the signal and noise arrays in `{ngc_galname}/{ngc_galname}_vorbin_SNR50_ugizBinned.pkl` (i.e., the data used to generate this file). Those have _NOT_ been normalized by the number of pixels per bin. "
                    + "\nThe relative magnitudes have been corrected for extinction; the binned signal and noise arrays have not. "
                    + "\n`ebv_coeff` is the extinction coefficient for the galaxy from the NGVS catalogue; "
                    + "`ebv_u`, `ebv_g`, `ebv_i`, `ebv_z` are the NGVS extinction coefficient conversion factors from http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/query/catdoc.html#ebv. "
                    + f"`isgood_snr` is a boolean mask indicating the pixels that have a finite SNR >= {good_snr} across all bands. "
                    + "`binNum_arr` is the 2D array of the same shape as the signal, noise, and magnitudes arrays indicating the bin number of each pixel. "
                    + "\nAlso remember to set `wcs_binned.array_shape = wcs_binned_array_shape`. "
                    + "\nIf `outside_to_closest` is True, assign values beyond the edges of the lookup table to the closest bin. If False, assign values beyond the edges of the lookup table to NaNs"
                    + "\n\nFinally, STELLAR MASS DENSITIES AND UNCERTAINTIES ARE ESTIMATED FROM OUR LOOKUP TABLE (see `coop_f2021/galaxies/lookup_table/TOTAL_no4189_extinctionCorr_lookupTable.pkl`) AND CORRECTED FOR INCLINATION",

                    "x_coords": x_coords,
                    "y_coords": y_coords,
                    "dist_pc": dist.to(u.pc).value, 
                    "inclination": inclination,
                    "wcs": wcs,
                    "wcs_array_shape": wcs.array_shape,
                    
                    "uband_signal": uband_signal,
                    "uband_noise": uband_noise,
                    "gband_signal": gband_signal,
                    "gband_noise": gband_noise,
                    "iband_signal": iband_signal,
                    "iband_noise": iband_noise,
                    "zband_signal": zband_signal,
                    "zband_noise": zband_noise,
                    
                    "isgood_snr": isgood_snr,
                    
                    "uband_rel_mag": uband_rel_mag,
                    "uband_rel_mag_err": uband_rel_mag_err,
                    "gband_rel_mag": gband_rel_mag,
                    "gband_rel_mag_err": gband_rel_mag_err,
                    "iband_rel_mag": iband_rel_mag,
                    "iband_rel_mag_err": iband_rel_mag_err,
                    "zband_rel_mag": zband_rel_mag,
                    "zband_rel_mag_err": zband_rel_mag_err,
                    
                    "px_per_bin": px_per_bin,
                    "binNum": binNum,
                    "binNum_arr": binNum_arr,
                    
                    "ebv_coeff": ebv_coeff,  # extinction coefficient for the galaxy
                    "ebv_u": EBV_u,  # extinction coefficient conversion factor for the u-band
                    "ebv_g": EBV_g,  # extinction coefficient conversion factor for the g-band
                    "ebv_i": EBV_i,  # extinction coefficient conversion factor for the i-band
                    "ebv_z": EBV_z,  # extinction coefficient conversion factor for the z-band
                    
                    "stellar_mass_density": M_density,
                    "stellar_mass_density_err": M_density_err,
                    "MLi": MLi,
                    "MLi_err": MLi_err,
                },
                f,
            )
        return galaxy_outfile
    
    def _regBin_to_M_density():
        gal_infile = galpath + f"{ngc_galname}_regBin_extinctionCorr_{bin_resolution}_noNorm.pkl"  # ! CHANGE ME
        with open(gal_infile, "rb") as f:
            file = dill.load(f)
            reproject_method = file["reproject_method"]
            binning_function = file["binning_function"]
            # dist_pc = file["dist_pc"]
            x_coords = file["x_coords"]
            y_coords = file["y_coords"]
            bin_dimensions = file["bin_dimensions"]
            wcs_binned = file["wcs_binned"]
            wcs_binned_array_shape = file["wcs_binned_array_shape"]
            ebv_coeff = file["ebv_coeff"]
            ebv_u = file["ebv_u"]
            ebv_g = file["ebv_g"]
            ebv_i = file["ebv_i"]
            ebv_z = file["ebv_z"]
            uband_signal = file["uband_signal"]
            uband_noise = file["uband_noise"]
            gband_signal = file["gband_signal"]
            gband_noise = file["gband_noise"]
            iband_signal = file["iband_signal"]
            iband_noise = file["iband_noise"]
            zband_signal = file["zband_signal"]
            zband_noise = file["zband_noise"]
            uband_rel_mag = file["uband_rel_mag"]
            uband_rel_mag_err = file["uband_rel_mag_err"]
            gband_rel_mag = file["gband_rel_mag"]
            gband_rel_mag_err = file["gband_rel_mag_err"]
            iband_rel_mag = file["iband_rel_mag"]
            iband_rel_mag_err = file["iband_rel_mag_err"]
            zband_rel_mag = file["zband_rel_mag"]
            zband_rel_mag_err = file["zband_rel_mag_err"]
            uband_is_good = file["uband_is_good"]
            gband_is_good = file["gband_is_good"]
            iband_is_good = file["iband_is_good"]
            zband_is_good = file["zband_is_good"]
            file = None  # free up memory
        #
        # Calculate stellar mass density (relative magnitudes already corrected for
        # extinction and fluxes already normalized)
        #
        gz_colour = gband_rel_mag - zband_rel_mag
        ui_colour = uband_rel_mag - iband_rel_mag
        MLi, MLi_err, M_density, M_density_err = assign_M_density(
            gz_colour, ui_colour, iband_rel_mag, iband_rel_mag_err, wcs_binned,
            inclination=inclination, dist=dist, outside_to_closest=outside_to_closest, print_debug=print_debug,
        )
        # 
        # Pickle data
        #
        galaxy_outfile = galpath + f"{ngc_galname}_regBin_extinctionCorr_{bin_resolution}_ugiz_Sigma-star_noNorm_i_corr.pkl"  # ! CHANGE ME
        readme_str = "The binned signal and noise arrays, as well as the relative magnitudes, have NOT been normalized by the number of pixels per bin. " \
                + "To normalize the binned signal and noise arrays, simply divide the signal and noise arrays by (`bin_dimensions`[0] * `bin_dimensions`[1]). " \
                + "\nThe relative magnitudes have been corrected for extinction; the binned signal and noise arrays have not. " \
                + "\n`ebv_coeff` is the extinction coefficient for the ngc_galname from the NGVS catalogue; " \
                + "`ebv_u`, `ebv_g`, `ebv_i`, `ebv_z` are the NGVS extinction coefficient conversion factors from http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/query/catdoc.html#ebv. " \
                + "\nAlso remember to set `wcs_binned.array_shape = wcs_binned_array_shape`. " \
                + "\nIf `outside_to_closest` is True, assign values beyond the edges of the lookup table to the closest bin. If False, assign values beyond the edges of the lookup table to NaNs" \
                + "\n\nFinally, STELLAR MASS DENSITIES AND UNCERTAINTIES ARE ESTIMATED FROM OUR LOOKUP TABLE (see `coop_f2021/galaxies/lookup_table/TOTAL_no4189_extinctionCorr_lookupTable.pkl`) AND CORRECTED FOR INCLINATION"
        with open(galaxy_outfile, "wb") as f:
            dill.dump(
                {
                    "README": readme_str,                    
                    "reproject_method": reproject_method,
                    "binning_function": binning_function,
                    "dist_pc": dist.to(u.pc).value,  # distance in parsecs
                    "inclination": inclination,
                    "x_coords": x_coords,  # x-value of pixel coordinates
                    "y_coords": y_coords,  # y-value of pixel coordinates
                    "bin_dimensions": bin_dimensions,
                    "wcs_binned": wcs_binned,  # dill has trouble saving the "NAXIS" keyword
                    "wcs_binned_array_shape": wcs_binned_array_shape,  # "NAXIS" keyword
                    "ebv_coeff": ebv_coeff,  # extinction coefficient for the galaxy
                    "ebv_u": ebv_u,  # extinction coefficient conversion factor for the u-band
                    "ebv_g": ebv_g,  # extinction coefficient conversion factor for the g-band
                    "ebv_i": ebv_i,  # extinction coefficient conversion factor for the i-band
                    "ebv_z": ebv_z,  # extinction coefficient conversion factor for the z-band
                    
                    "uband_signal": uband_signal,
                    "uband_noise": uband_noise,
                    "gband_signal": gband_signal,
                    "gband_noise": gband_noise,
                    "iband_signal": iband_signal,
                    "iband_noise": iband_noise,
                    "zband_signal": zband_signal,
                    "zband_noise": zband_noise,

                    "uband_rel_mag": uband_rel_mag,
                    "uband_rel_mag_err": uband_rel_mag_err,
                    "gband_rel_mag": gband_rel_mag,
                    "gband_rel_mag_err": gband_rel_mag_err,
                    "iband_rel_mag": iband_rel_mag,
                    "iband_rel_mag_err": iband_rel_mag_err,
                    "zband_rel_mag": zband_rel_mag,
                    "zband_rel_mag_err": zband_rel_mag_err,
                    
                    "uband_is_good": uband_is_good,
                    "gband_is_good": gband_is_good,
                    "iband_is_good": iband_is_good,
                    "zband_is_good": zband_is_good,

                    "stellar_mass_density": M_density,
                    "stellar_mass_density_err": M_density_err,
                    "MLi": MLi,
                    "MLi_err": MLi_err,
                    "outside_to_closest": outside_to_closest,
                },
                f,
            )
        return galaxy_outfile
    
    if print_debug:
        print(f"On {ngc_galname} (VCC {vcc_galnum}) at {bin_resolution} resolution")
    galpath = f"/arc/home/IsaacCheng/coop_f2021/galaxies/{ngc_galname}/"  # ! CHANGE ME
    if bin_resolution == "vorbin":
        gal_outfile = _vorbin_to_M_density()
    elif bin_resolution == "nyquist" or bin_resolution == "9as":
        gal_outfile = _regBin_to_M_density()
    else:
        raise ValueError("bin_resolution must be either 'vorbin', 'nyquist', or '9as'.")
    print("Pickled", gal_outfile)


## 5. Run function

Again, for each galaxy, we will use the function above to produce a pickle file containing
all the raw + derived data for this galaxy. The resulting pickle file for this galaxy will
be the basis for pretty much all of the analysis involving optical data or their derived
products.

Once the "nyquist" resolution is done, proceed to the next resolution by commenting out
the current line and un-commenting the next line.

Please make sure you have enough memory to do this in parallel!


In [9]:
with Pool(processes=None) as p:
    p.starmap(ngvs_to_MLi, zip(GALAXIES, VCC_GALAXIES, INCLINATIONS, repeat("nyquist")))
    # p.starmap(ngvs_to_MLi, zip(GALAXIES, VCC_GALAXIES, INCLINATIONS, repeat("9as")))
    # p.starmap(ngvs_to_MLi, zip(GALAXIES, VCC_GALAXIES, INCLINATIONS, repeat("vorbin")))
print("Done")

WARNING
: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]WARNING

: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]WARNING
WARNING








Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/IC3392/IC3392_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4405/NGC4405_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl


/arc/home/IsaacCheng/coop_f2021/packages/fits_utils.py:241: RuntimeWarning: invalid value encountered in log10
  rel_mag = -2.5 * np.log10(flux) + zpt
/arc/home/IsaacCheng/coop_f2021/packages/fits_utils.py:241: RuntimeWarning: invalid value encountered in log10
  rel_mag = -2.5 * np.log10(flux) + zpt


Pickled

: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]


...

/arc/home/IsaacCheng/coop_f2021/galaxies/NGC4568/NGC4568_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pklPickled/arc/home/IsaacCheng/coop_f2021/galaxies/NGC4419/NGC4419_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
 Pickled/arc/home/IsaacCheng/coop_f2021/galaxies/NGC4351/NGC4351_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl 

Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4694/NGC4694_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4383/NGC4383_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4222/NGC4222_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4396/NGC4396_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled Pickled/arc/home/IsaacCheng/coop_f2021/galaxies/NGC4567/NGC4567_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
 /arc/home/IsaacCheng/coop_f2021/galaxies

/arc/home/IsaacCheng/coop_f2021/packages/fits_utils.py:241: RuntimeWarning: invalid value encountered in log10
  rel_mag = -2.5 * np.log10(flux) + zpt


Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4651/NGC4651_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4330/NGC4330_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl


/arc/home/IsaacCheng/coop_f2021/packages/fits_utils.py:241: RuntimeWarning: invalid value encountered in log10
  rel_mag = -2.5 * np.log10(flux) + zpt


Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4380/NGC4380_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4298/NGC4298_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4450/NGC4450_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4388/NGC4388_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4501/NGC4501_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4569/NGC4569_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4548/NGC4548_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/NGC4689/NGC4689_vorbin_SNR50_extinctionCorr_ugiz_Sigma-star_i_corr.pkl
Pickled /arc/home/IsaacCheng/coop_f2021/